In [7]:
#George Jen, Jen Tek LLC

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession


In [9]:
#spark = SparkSession.builder.getOrCreate()
#Read Data
#df = spark.read.csv("datasets/SMSSpamCollection", sep = "\t", inferSchema=True, header = False)

In [10]:
#Logger.getLogger("org").setLevel(Level.ERROR)
spark = SparkSession \
    .builder \
    .appName("csv2parquet") \
    .master("local[*]") \
    .config("spark.sql.warehouse.dir", "file:///d:/tmp") \
    .getOrCreate()


In [12]:
sc=spark.sparkContext

In [13]:
sc.setLogLevel("ERROR")

In [14]:
ds = spark.read.format("csv").option("header", "true").option("quote", "\"").load("D:/teaching/scala/ticker_symbol.csv")


In [18]:
ds.show(2,False)

+------+----------------------------+--------+---------------------------+--------------+----+
|Ticker|Name                        |Exchange|CategoryName               |CategoryNumber|_c5 |
+------+----------------------------+--------+---------------------------+--------------+----+
|AUB.AX|Austbrokers Holdings Limited|ASX     |Accident & Health Insurance|431           |null|
|GLRE  |Greenlight Capital Re, Ltd. |NMS     |Accident & Health Insurance|431           |null|
+------+----------------------------+--------+---------------------------+--------------+----+
only showing top 2 rows



In [25]:
heading = ["Ticker","Name","Exchange","CategoryName","CategoryNumber","_c5"]

In [27]:
df = ds.toDF(*heading)

In [28]:
df.show(3, False)

+------+-----------------------------+--------+---------------------------+--------------+----+
|Ticker|Name                         |Exchange|CategoryName               |CategoryNumber|_c5 |
+------+-----------------------------+--------+---------------------------+--------------+----+
|AUB.AX|Austbrokers Holdings Limited |ASX     |Accident & Health Insurance|431           |null|
|GLRE  |Greenlight Capital Re, Ltd.  |NMS     |Accident & Health Insurance|431           |null|
|SFG   |StanCorp Financial Group Inc.|NYQ     |Accident & Health Insurance|431           |null|
+------+-----------------------------+--------+---------------------------+--------------+----+
only showing top 3 rows



In [30]:
#When the CSV file was read into DataFrame, all fields are String, below is to cast it to
#what the data should be, such as cast CategoryNumber to Int

df_with_datatype=df.selectExpr("Ticker",\
                  "Name", \
                  "Exchange",\
                  "CategoryName",\
                  "cast(CategoryNumber as int) CategoryNumber")

df_with_datatype.show(3, False)

+------+-----------------------------+--------+---------------------------+--------------+
|Ticker|Name                         |Exchange|CategoryName               |CategoryNumber|
+------+-----------------------------+--------+---------------------------+--------------+
|AUB.AX|Austbrokers Holdings Limited |ASX     |Accident & Health Insurance|431           |
|GLRE  |Greenlight Capital Re, Ltd.  |NMS     |Accident & Health Insurance|431           |
|SFG   |StanCorp Financial Group Inc.|NYQ     |Accident & Health Insurance|431           |
+------+-----------------------------+--------+---------------------------+--------------+
only showing top 3 rows



In [32]:
#Save the DataFrame to Parquet format, overwrite if existing.
#Parquet is Columnar, good for Analytics query.

df_with_datatype.write.mode("Overwrite").parquet("D:/teaching/scala/ticker_symbol.parquet")

In [33]:
#Read the Parquet data back and run SQL query on it

read_parquet_df = spark.read.parquet("D:/teaching/scala/ticker_symbol.parquet")

read_parquet_df.show(3, False)

+------+-----------------------------+--------+---------------------------+--------------+
|Ticker|Name                         |Exchange|CategoryName               |CategoryNumber|
+------+-----------------------------+--------+---------------------------+--------------+
|AUB.AX|Austbrokers Holdings Limited |ASX     |Accident & Health Insurance|431           |
|GLRE  |Greenlight Capital Re, Ltd.  |NMS     |Accident & Health Insurance|431           |
|SFG   |StanCorp Financial Group Inc.|NYQ     |Accident & Health Insurance|431           |
+------+-----------------------------+--------+---------------------------+--------------+
only showing top 3 rows



In [35]:
TickerSymbol = read_parquet_df.toDF("Ticker","Name","Exchange","CategoryName","CategoryNumber")
TickerSymbol.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Exchange: string (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- CategoryNumber: integer (nullable = true)



In [36]:
TickerSymbol.createOrReplaceTempView("TickerSymbol")
spark.sql("SELECT * from TickerSymbol where Ticker in ('IBM','MSFT','HPQ','GE')").show(20,False)

+------+-------------------------------------------+--------+-------------------------------+--------------+
|Ticker|Name                                       |Exchange|CategoryName                   |CategoryNumber|
+------+-------------------------------------------+--------+-------------------------------+--------------+
|MSFT  |Microsoft Corporation                      |NMS     |Business Software & Services   |826           |
|HPQ   |Hewlett-Packard Company                    |NYQ     |Diversified Computer Systems   |810           |
|GE    |General Electric Company                   |NYQ     |Diversified Machinery          |622           |
|IBM   |International Business Machines Corporation|NYQ     |Information Technology Services|824           |
+------+-------------------------------------------+--------+-------------------------------+--------------+

